In [ ]:
# 진행 단계: 모든 파일은 코드 확인 및 수정 후 실행할 것
# 1. data_aug.ipynb -> 2. adding_special_token.ipynb -> 3. separate_dev_data.ipynb -> 
# 4. config.yaml 수정 -> 5. train.py -> 6. inference.py -> 7. ensemble.ipynb

# 노트북 파일을 dataset 폴더에서 code 폴더로 옮겼기에, 데이터 경로를 바꿔야 함

label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']

In [ ]:
import pandas as pd

df_num = 1  # 바꿔야 할 숫자
# 베이스 데이터프레임
df1 = pd.read_csv("../code/prediction/submission_aug_copy_typed_entity_marker_punc_10portions_01.csv").drop(['pred_label'], axis=1)
df1.rename(columns = {'probs': 'probs1'}, inplace = True)
# 각 예측 확률값을 베이스 데이터프레임에 추가함
for i in range(2, df_num+1):
    temp_df = pd.read_csv("../code/prediction/submission_aug_copy_typed_entity_marker_punc_10portions_{0:02d}.csv".format(i)).drop(['pred_label'], axis=1)
    temp_df.rename(columns = {'probs': 'probs{0}'.format(i)}, inplace = True)
    df1 = pd.merge(left = df1 , right = temp_df, how = "inner",)
df1

In [ ]:
probs_list = []  # index별 확률분포
pred_label_list = []  # index별 예상 라벨
from ast import literal_eval
for idx, row in df1.iterrows():
    # soft-voting 하기
    probs = [0 for _ in range(len(literal_eval(row['probs1'])))]  # 라벨별 확률 리스트 초기화
    for i in range(1,df_num+1):
        temp_probs = literal_eval(row['probs{0}'.format(i)]) # type==list
        for j in range(len(probs)):
            #probs[j] += temp_probs[j]/df_num  # soft-voting(라벨별 평균 확률)을 내기 위해 라벨별 확률을 데이터프레임 수만큼 나눈 후 probs 리스트에 추가
            probs[j] += temp_probs[j]/(df_num-1)
    
    # 인덱스별로 확률이 가장 높은 라벨 구하기
    max_i = 0
    max_prob = 0
    for i in range(len(probs)):
        if max_prob < probs[i]:
            max_prob = probs[i]
            max_i = i
    probs_list.append(probs)
    pred_label_list.append(label_list[max_i])

# 제출물 포맷에 맞게 컬럼 추가하기
df1['pred_label']=pred_label_list
df1['probs']=probs_list
df1 = df1.drop(['probs{}'.format(i) for i in range(1,df_num+1)], axis=1)  # 이미 써서 쓸모 없는 컬럼은 제거
df1.to_csv("../code/prediction/submission_aug_copy_typed_entity_marker_punc_10portions_01-{0:02d}.csv".format(df_num), index=False)
df1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv("../code/prediction/submission_aug_copy_typed_entity_marker_punc_10portions_01-{0:02d}.csv".format(df_num), index=False)
plot = pd.value_counts(df['pred_label'].values, sort=True)
plot.plot.bar()